<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>


This notebook contains the logic used to perform calculations upon import of data (actuals and cash flows). This is also called 'Data Loading' and the concept of Scope is used here to define the logic and provide the means of executing the logic. Scopes are used to define and perform data handling in a structured and easy-to-read-through fashion.

<br><br>
Import scopes are divided in several notebooks:
- Calculation of Identities
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- [Calculation of Actuals](../Import/3ImportScope-Actuals)
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- [Creation of Ifrs Variables](../Import/5ImportScope-ToIfrsVar)
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the defintion of the Identities and on the AoC Step structure and relations required for the computations. 


# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "ImportStorage"

# IModel 

The <code>IModel</code> interface below will be used to execute calculations (i.e. evaluate the scopes) based on imported data.

In [0]:
public interface IModel : IMutableScopeWithStorage<ImportStorage>{}

# Generate Identities

An [Identity](../DataModel/DataStructure#import-identity) is a set of identifiers for a certain set of data. In particular, an identity consists of a certain [AoC Type](../DataModel/DataStructure#aoc-type), [Novelty](../DataModel/DataStructure#novelty), [Data Node](../DataModel/DataStructure#data-node), Accident Year, and information on whether the data is for reinsurance or not. 

Given a certain Data Node and Accident Year, the interface <code>GetIdentities</code> returns all the existing identities (e.g. for Actuals and Cash flows) which have that Data Node and Accident Year.

In [0]:
public interface AllCfIdentities : IScope<string, ImportStorage> // Identity corresponds to the DataNode
{
    IEnumerable<ImportIdentity> ids => GetStorage().GetAllAocSteps(InputSource.Cashflow)
        .Select(aocStep => new ImportIdentity {AocType = aocStep.AocType, Novelty = aocStep.Novelty, DataNode = Identity });
}

In [0]:
public interface GetParsedAocSteps : IScope<string, ImportStorage>
{
    IEnumerable<AocStep> Values => GetStorage().GetRawVariables(Identity).Select(x => new AocStep(x.AocType, x.Novelty)).Distinct();
}

In [0]:
public interface GetIdentities : IScope<string, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<GetIdentities>(s => s.WithApplicability<GetCashflowIdentities>(x => x.GetStorage().ImportFormat == ImportFormats.Cashflow));
    
    protected IEnumerable<ImportIdentity> allIdentities => GetStorage().AocConfigurationByAocStep.Values.Select(x => new ImportIdentity {AocType = x.AocType, Novelty = x.Novelty, DataNode = Identity });

    IEnumerable<ImportIdentity> Identities => allIdentities.Select(id => id with { IsReinsurance = GetStorage().DataNodeDataBySystemName[id.DataNode].IsReinsurance,
                                                                                   ValuationApproach = GetStorage().DataNodeDataBySystemName[id.DataNode].ValuationApproach});
}

public interface GetCashflowIdentities : GetIdentities
{
    private bool isReinsurance => GetStorage().DataNodeDataBySystemName[Identity].IsReinsurance;
    
    private IEnumerable<ImportIdentity> ParsedIdentities => GetScope<GetParsedAocSteps>(Identity).Values.Select(aocStep => new ImportIdentity {AocType = aocStep.AocType, Novelty = aocStep.Novelty, DataNode = Identity});
    
    private IEnumerable<string> rawVariableNovelties => GetStorage().GetRawVariables(Identity).Select(rv => rv.Novelty).Distinct().Concat(Novelties.C.RepeatOnce());

    private IEnumerable<AocStep> calculatedAocSteps => GetStorage().AocConfigurationByAocStep.Values.Where(x => new DataType[]{DataType.Calculated, DataType.CalculatedTelescopic}.Contains(x.DataType) &&
        (!isReinsurance ? !new []{AocTypes.CRU, AocTypes.RCU}.Contains(x.AocType) : true) && rawVariableNovelties.Contains(x.Novelty) ).Select(x => new AocStep(x.AocType, x.Novelty));

    private IEnumerable<ImportIdentity> SpecialIdentities => calculatedAocSteps.Select(x => new ImportIdentity {AocType = x.AocType, Novelty = x.Novelty, DataNode = Identity })
        .Concat(new ImportIdentity {AocType = AocTypes.CF, Novelty = Novelties.C,DataNode = Identity}.RepeatOnce()) //Add CF for Deferral
        .Concat(GetStorage().GetAllAocSteps(InputSource.Opening).Select(aocStep => new ImportIdentity {AocType = aocStep.AocType, Novelty = aocStep.Novelty, DataNode = Identity}));//Add BOP,I and IA,I

    IEnumerable<ImportIdentity> GetIdentities.allIdentities => ParsedIdentities.Concat(SpecialIdentities).Distinct();       
}

# Relevant Amount Types

Similarly, given a certain Data Node and Accident Year, the interface <code>ValidAmountType</code> returns all the amount types which are used in imported data with that Data Node and Accident Year.

In [0]:
public interface ValidAmountType : IScope<string, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
                builder.ForScope<ValidAmountType>(s => s.WithApplicability<BeAmountTypesFromIfrsVariables>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow ||
                                                                                                                x.GetStorage().IsSecondaryScope(x.Identity)));
    
    IEnumerable<string> BeAmountTypes => GetStorage().GetRawVariables(Identity)
                                            .Where(rv => rv.AmountType != null)
                                            .Select(x => x.AmountType)
                                            .Concat(GetStorage().DataNodeDataBySystemName[Identity].IsReinsurance 
                                                    ? GetStorage().DataNodeDataBySystemName[Identity].LiabilityType == LiabilityTypes.LIC 
                                                        ? new []{AmountTypes.CDR} : new []{AmountTypes.CDRI, AmountTypes.CDR} 
                                                    : Enumerable.Empty<string>()).ToHashSet();
    
    
    IEnumerable<string> ActualAmountTypes => GetStorage().GetIfrsVariables(Identity)
                                                         .Where(iv => GetStorage().ImportActualEstimateTypes.Contains(iv.EstimateType))
                                                         .Select(x => x.AmountType)
                                                         .ToHashSet();
}
public interface BeAmountTypesFromIfrsVariables : ValidAmountType
{
    IEnumerable<string> ValidAmountType.BeAmountTypes => GetStorage().GetIfrsVariables(Identity)
                                                         .Where(iv => GetStorage().EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(iv.EstimateType) && iv.AmountType != null)
                                                         .Select(x => x.AmountType)
                                                         .ToHashSet();
}

# AoC Step Structure

The AoC Step structure is constructed from the data which is delivered as input. It is assumed that it depends only on the Group of Contract (i.e. it is invariant across Amount types or Accident Year). 

## Parent AoC Step 
The **ParentAocStep** scope computes and provides an important piece of information for each [Identity](../DataModel/DataStructure#import-identity), i.e. for a certain [AoC Type](../DataModel/DataStructure#aoc-type) and [Novelty](../DataModel/DataStructure#novelty).

It provides the list of the adjacent AoC Steps prior to Identity one. It can be more than one only for the step **CL** where a parent for each novelty is considered.
ParentAocStep is critical when computing the *telescoping* differences. 

They are defined as follows:

$$
\text{ParentAocStep}(\text{AoC Step}) = \left\{
\begin{array}{cl}
\text{AoC Step with AoC Type YCU and Novelty I} & \text{if AoC Type CRU} \\
\text{The last AoC Step with Data Type != Calculated and same Novelty as the AoC Step} & \text{if AoC Type YCU} \\
\text{The AoC Step which comes before in terms of order (as defined by AoC Type)} & \text{otherwise} \\
\end{array}
\right.
$$


In [0]:
public interface ParentAocStep : IScope<(ImportIdentity Id, string AmountType), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
                builder.ForScope<ParentAocStep>(s => s.WithApplicability<ParentAocStepForCreditRisk>(x => x.Identity.AmountType != AmountTypes.CDR));
    
    private IEnumerable<AocStep> ParsedAocSteps => GetScope<GetParsedAocSteps>(Identity.Id.DataNode).Values;
    private IEnumerable<AocStep> OrderedParsedAocSteps => ParsedAocSteps.Concat(CalculatedTelescopicAocStep).OrderBy(x => GetStorage().AocConfigurationByAocStep[x].Order);
    
    private Dictionary<AocStep,IEnumerable<AocStep>> ParentParsedIdentities => GetPreviousIdentities(OrderedParsedAocSteps);
    private AocStep identityAocStep => new AocStep(Identity.Id.AocType, Identity.Id.Novelty);
    
    IEnumerable<AocStep> CalculatedTelescopicAocStep => GetStorage().GetCalculatedTelescopicAocSteps();
    
    IEnumerable<AocStep> Values => 
        Identity.Id.AocType switch {
                 AocTypes.CRU => new AocStep(AocTypes.YCU, Novelties.I).RepeatOnce(),
                 AocTypes.YCU => OrderedParsedAocSteps.GetReferenceAocStepForCalculated(GetStorage().AocConfigurationByAocStep, identityAocStep).RepeatOnce(),
                 _ => ParentParsedIdentities.TryGetValue(identityAocStep, out var parents) ? parents : Enumerable.Empty<AocStep>(),
    };
}

public interface ParentAocStepForCreditRisk : ParentAocStep
{
   IEnumerable<AocStep> ParentAocStep.CalculatedTelescopicAocStep => 
       GetStorage().GetCalculatedTelescopicAocSteps().Where(aoc => aoc.AocType != AocTypes.CRU);
}

## Reference AoC Step

The **ReferenceAocStep** scope provides the AoC Step from which to retrieve the data in order to compute its value (e.g. AoC Step EA retrieves Present Values, while AoC Step YCU retrieves Nominal).

They are defined as follows:

$$
\text{ReferenceAocStep}(\text{AoC Step}) = \left\{
\begin{array}{cl}
\text{self} & \text{if AoC Step InputSource is not Calculated} \\
\text{The last AoC Step with Data Type != Calculated and same Novelty as the input AoC Step} ~, & \text{if AoC Type } \in \text{\{RCU, CF, IA, YCU, CRU\}} \\
\text{AoC Step with AoC Type CF and Novelty as the AoC Step} & \text{if AoC Type EA} \\
\text{AoC Step with AoC Type CL and Novelty C} & \text{if AoC Type $\in$ \{AM, EOP\}} \\
\text{empty} & \text{if AoC Type is BOP} \\
\text{log NotSupportedAocStepReference error} & \text{otherwise} \\
\end{array}
\right.
$$

where the last AoC Step is obtained by ordering the AoC Steps according to their order (as defined by its AoC Type) and taking the last one.

In [0]:
public interface ReferenceAocStep : IScope<ImportIdentity, ImportStorage> 
{
    private IEnumerable<AocStep> OrderedParsedAocSteps => GetScope<GetParsedAocSteps>(Identity.DataNode).Values.OrderBy(aocStep => GetStorage().AocConfigurationByAocStep[aocStep].Order);
    private AocStep identityAocStep => new AocStep(Identity.AocType, Identity.Novelty);
    private AocStep GetReferenceAocStep(string aocType) {
        return aocType switch {
               AocTypes.RCU or AocTypes.CF or AocTypes.IA or AocTypes.YCU or AocTypes.CRU => OrderedParsedAocSteps.GetReferenceAocStepForCalculated(GetStorage().AocConfigurationByAocStep, identityAocStep),
               AocTypes.EA => new AocStep(AocTypes.CF, Identity.Novelty),
               AocTypes.AM or AocTypes.EOP => new AocStep(AocTypes.CL, Novelties.C),
               AocTypes.BOP => GetStorage().GetShift(Identity.ProjectionPeriod) >= MonthInAYear && Identity.Novelty == Novelties.I ? new AocStep(AocTypes.CL, Novelties.C) : new AocStep(default, default), //BOP, C has DataType == Calculated. See ReferenceAocStep condition.
               _ => (AocStep)ApplicationMessage.Log(Error.NotSupportedAocStepReference, Identity.AocType),
        };
    }

    // The Reference AocStep from which get data (Nominal or PV) to compute
    AocStep Value => GetStorage().AocConfigurationByAocStep[identityAocStep].DataType == DataType.Calculated 
                     || GetStorage().AocConfigurationByAocStep[identityAocStep].DataType == DataType.CalculatedTelescopic 
                                                ? GetReferenceAocStep(Identity.AocType)
                                                : identityAocStep;
}

## Previous AoC Steps
The **PreviousAocSteps** scope computes and provides another important piece of information for each [Identity](../DataModel/DataStructure#import-identity), i.e. for a certain [AoC Type](../DataModel/DataStructure#aoc-type) and [Novelty](../DataModel/DataStructure#novelty).

It provides the list of all previous AoC Steps up to the **BOP** step, whereby a Combined novelty will branch into the InForce and  New Business AoC Types.

PreviousAocSteps is critical when computing aggregated values along the various dimensions (such as for example Line of Business) and 
is formed by the ParentAocStep and its parent and so on until there is no parent.

$$
\text{PreviousAocSteps}(\rm{AocStep}) = \{PAS_1, PAS_2, \ldots\}
$$
where
$$
\rm{PAS}_1 = \rm{ParentAocStep}(\rm{AoC Step})
$$
$$
\rm{PAS}_2 = \rm{ParentAocStep}(\rm{PAS}_1).
$$

This scope depends on the InputSource (Actual or Cashflow) for which the PreviousSteps are requested due to the AocChain differences between Actual reports and the rest.

In [0]:
public interface PreviousAocSteps : IScope<(ImportIdentity Id, InputSource ScopeInputSource), ImportStorage> 
{   
    private AocStep identityAocStep => new AocStep(Identity.Id.AocType, Identity.Id.Novelty);
    private int aocStepOrder => GetStorage().AocConfigurationByAocStep[identityAocStep].Order;
    private HashSet<AocStep> allAocSteps => GetStorage().GetAllAocSteps(Identity.ScopeInputSource).ToHashSet();
    IEnumerable<AocStep> Values => allAocSteps.Contains(identityAocStep)
                                        ? GetScope<GetIdentities>(Identity.Id.DataNode).Identities
                                                                 .Select(id => new AocStep(id.AocType, id.Novelty))
                                                                 .Distinct()
                                                                 .Where(aoc => allAocSteps.Contains(aoc) && 
                                                                               GetStorage().AocConfigurationByAocStep[aoc].Order < aocStepOrder && 
                                                                               (Identity.Id.Novelty != Novelties.C ? aoc.Novelty == Identity.Id.Novelty : true) )
                                                                 .OrderBy(aoc => GetStorage().AocConfigurationByAocStep[aoc].Order)
                                        : Enumerable.Empty<AocStep>();
} 

The exact structure being return depends on the **order** of the AoC Steps (which is set by the [AoC Type](../DataModel/DataStructure#aoc-type)), and on which AoC steps exist.